In [2]:
!mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [3]:
!cp "/content/drive/MyDrive/Hackathons/kaggle/kaggle.json" ~/.kaggle/kaggle.json

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!python -m pip install -qq kaggle

In [6]:
!kaggle competitions download -c nlp-getting-started

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip -qq "/content/nlp-getting-started.zip"

replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [8]:
import pandas as pd

In [9]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [10]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
len(train_df), len(test_df)

(7613, 3263)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df["text"].to_numpy(),
    train_df["target"].to_numpy(),
    test_size = 0.2,
    random_state=42
)

In [15]:
train_sentences[:10]

array(['Courageous and honest analysis of need to use Atomic Bomb in 1945. #Hiroshima70 Japanese military refused surrender. https://t.co/VhmtyTptGR',
       '@ZachZaidman @670TheScore wld b a shame if that golf cart became engulfed in flames. #boycottBears',
       "Tell @BarackObama to rescind medals of 'honor' given to US soldiers at the Massacre of Wounded Knee. SIGN NOW &amp; RT! https://t.co/u4r8dRiuAc",
       'Worried about how the CA drought might affect you? Extreme Weather: Does it Dampen Our Economy? http://t.co/fDzzuMyW8i',
       '@YoungHeroesID Lava Blast &amp; Power Red #PantherAttack @JamilAzzaini @alifaditha',
       "Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/yi54XRHQGB",
       'Our builder is having a dental emergency. Which has ruined my plan to emotionally blackmail him this afternoon with my bump.',
       'BMX issues Areal Flood Advisory for Shelby [AL] till Aug 5 9:00 PM CDT htt

In [16]:
train_labels[:10]

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 0])

In [17]:
test_sentences = test_df["text"].to_numpy()

In [18]:
test_sentences[:5]

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       'Apocalypse lighting. #Spokane #wildfires',
       'Typhoon Soudelor kills 28 in China and Taiwan'], dtype=object)

In [19]:
import tensorflow as tf

In [20]:
from tensorflow.keras.layers import TextVectorization

In [21]:
round(sum([len(i.split()) for i in train_sentences]) / len(train_labels))

15

In [22]:
max_vocal_length = 10000
max_length = 30

In [23]:
text_vectorizer = TextVectorization(
    max_tokens= max_vocal_length,
    output_mode = "int",
    output_sequence_length = max_length
)

In [24]:
text_vectorizer.adapt(train_sentences)

In [25]:
sample_sentence = "Elon Musk Is A Billaniours"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 30), dtype=int64, numpy=
array([[   1, 9737,    9,    3,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])>

In [26]:
word_in_verb = text_vectorizer.get_vocabulary()

In [27]:
word_in_verb[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'i', 'and', 'is']

In [28]:
word_in_verb[-10:]

['midsouth',
 'midmorning',
 'midget',
 'midges',
 'midfield',
 'mideast',
 'middleeasteye',
 'midday',
 'microwave',
 'microphone']

In [29]:
from tensorflow.keras import layers

In [30]:
embedding = layers.Embedding(
    input_dim = max_vocal_length,
    output_dim = 128,
    input_length = max_length
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [31]:
embedding(text_vectorizer([sample_sentence]))

<tf.Tensor: shape=(1, 30, 128), dtype=float32, numpy=
array([[[-0.00940534,  0.04781445,  0.01127909, ..., -0.01694291,
         -0.00775223,  0.0266774 ],
        [ 0.01389122, -0.01466556, -0.00646896, ...,  0.04738492,
          0.04206988, -0.02384359],
        [ 0.03132881, -0.04819785, -0.00172597, ..., -0.0072718 ,
         -0.03224996, -0.01618903],
        ...,
        [ 0.01547854, -0.00445045, -0.02461795, ...,  0.04439414,
         -0.02848544, -0.00814915],
        [ 0.01547854, -0.00445045, -0.02461795, ...,  0.04439414,
         -0.02848544, -0.00814915],
        [ 0.01547854, -0.00445045, -0.02461795, ...,  0.04439414,
         -0.02848544, -0.00814915]]], dtype=float32)>

In [59]:
model = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype = tf.string),
    text_vectorizer,
    embedding,
    layers.Dropout(0.5),
    layers.LSTM(32),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [60]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization                   │ (None, 30)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 30, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 30, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 32)                  │          20,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,301,153 (4.96 MB)

 Trainable params: 1,301,153 (4.96 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"],
)

In [62]:
history = model.fit(
    train_sentences,
    train_labels,
    epochs =5,
    validation_data = (val_sentences, val_labels)
)

Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.7838 - loss: 0.4187 - val_accuracy: 0.7689 - val_loss: 0.8516
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.9746 - loss: 0.1175 - val_accuracy: 0.7722 - val_loss: 0.8219
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9789 - loss: 0.0808 - val_accuracy: 0.7708 - val_loss: 0.9692
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9826 - loss: 0.0542 - val_accuracy: 0.7643 - val_loss: 1.1454
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9807 - loss: 0.0477 - val_accuracy: 0.7485 - val_loss: 1.3696


In [63]:
model_pred_probs = model.predict(val_sentences)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


In [64]:
model_pred_probs[:5]

array([[0.00179   ],
       [0.9992724 ],
       [0.00851435],
       [0.00138953],
       [0.9994109 ]], dtype=float32)

In [65]:
model_preds = tf.squeeze(tf.round(model_pred_probs))

In [66]:
model_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 0., 0., 1.], dtype=float32)>

In [67]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

  model_result = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1-score": model_f1
      }

  return model_result

In [68]:
model_2_results = calculate_results(val_labels, model_preds)

In [69]:
model_2_results

{'accuracy': 0.7485226526592252,
 'precision': 0.7537339128846416,
 'recall': 0.7485226526592252,
 'f1-score': 0.7496828460723337}

In [76]:
model_preds = model.predict(test_sentences)

102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [81]:
model_preds = tf.squeeze(tf.round(model_preds))

In [82]:
sample = pd.read_csv('sample_submission.csv')

In [83]:
sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [84]:
submissions = pd.DataFrame(columns=["id","target"])
submissions["target"] = model_preds
submissions["id"] = sample.id

In [85]:
submissions.head()

,id,target
0,0,0.0
1,2,1.0
2,3,1.0
3,9,1.0
4,11,1.0


In [87]:
submissions.to_csv("submission.csv", index=False)